**1. Initialization**

In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install -q tensorflow==2.0.0-beta1
!pip install "numpy<1.17"

import numpy as np
import tensorflow as tf

print("TF", tf.__version__)
print("NP", np.__version__)


TF 2.0.0-beta1
NP 1.18.4


**2. Training/test Dataset**

In [12]:
X = tf.constant([[1, 2, 1], 
                 [1, 3, 2], 
                 [1, 3, 4], 
                 [1, 5, 5], 
                 [1, 7, 5], 
                 [1, 2, 5],
                 [1, 6, 6],
                 [1, 7, 7]], dtype=tf.float32, shape=[8, 3])    
Y = tf.constant([[0, 0, 1], 
                 [0, 0, 1], 
                 [0, 0, 1], 
                 [0, 1, 0], 
                 [0, 1, 0], 
                 [0, 1, 0],
                 [1, 0, 0],
                 [1, 0, 0]], dtype=tf.float32, shape=[8, 3])

# If input data was too different, add normalized function to normalize the input data

class_num = 3    

W = tf.Variable(tf.random.uniform(shape=[3, class_num], name='weight', minval=0, maxval=1, dtype=tf.float32, seed=3))
b = tf.Variable(tf.random.uniform(shape=[class_num], name='bias', minval=0, maxval=1, dtype=tf.float32, seed=3))

def hypothesis(x, w, b):
    return tf.nn.softmax(tf.linalg.matmul(x, w) + b)

def cost(x, w, b, y):
    H = hypothesis(x, w, b)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(H), axis=1))

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1) # If learning_rate is too big, cost become bigger
                                                        # If learning_rate is too small, cost become jump into local minimum
loss = lambda: cost(X, W, b, Y) 

for step in range(2001):
    optimizer.minimize(loss, var_list=[W, b])
    if step % 200 == 0:
        print(step, cost(X, W, b, Y))

0 tf.Tensor(1.2268403, shape=(), dtype=float32)
200 tf.Tensor(0.6005517, shape=(), dtype=float32)
400 tf.Tensor(0.4987751, shape=(), dtype=float32)
600 tf.Tensor(0.43992436, shape=(), dtype=float32)
800 tf.Tensor(0.3981998, shape=(), dtype=float32)
1000 tf.Tensor(0.36625576, shape=(), dtype=float32)
1200 tf.Tensor(0.34067917, shape=(), dtype=float32)
1400 tf.Tensor(0.3195417, shape=(), dtype=float32)
1600 tf.Tensor(0.30164364, shape=(), dtype=float32)
1800 tf.Tensor(0.28619272, shape=(), dtype=float32)
2000 tf.Tensor(0.2726435, shape=(), dtype=float32)


In [13]:
X_test = tf.constant([[2, 1, 1],
                       [3, 1, 2],
                       [3, 3, 4]], dtype=tf.float32, shape=[3, 3])
Y_test = tf.constant([[0, 0, 1],
                       [0, 0, 1],
                       [0, 0, 1]], dtype=tf.float32, shape=[3, 3])

def prediction(h):
    return tf.math.argmax(h, 1)

def is_correct(pred, y):
    return tf.equal(pred, tf.math.argmax(y, 1))

def accuracy(correct_pred):
    return tf.reduce_mean(tf.cast(correct_pred, tf.float32))

hypo = hypothesis(X_test, W, b)
print("Prediction: {}".format(prediction(hypo)))
acc = accuracy(is_correct(prediction(hypo), Y_test))
print("Accuracy: {}".format(acc))


Prediction: [2 2 2]
Accuracy: 1.0


In [17]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(
    3, # output_shape
    input_shape=(3, ),
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='glorot_uniform',
    activation='softmax'))

model.summary()

model.compile(
    optimizer='SGD',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

history = model.fit(
    x=X,
    y=Y,
    # batch_size=3,
    epochs=2000,
    validation_data=(X_test, Y_test),
    verbose=0
)

test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=2)

predictions = model.predict(X_test)
print("pred: {}, actual: {}".format(np.argmax(predictions[0]), np.argmax(Y_test[0])))
print("pred: {}, actual: {}".format(np.argmax(predictions[1]), np.argmax(Y_test[1])))
print("pred: {}, actual: {}".format(np.argmax(predictions[2]), np.argmax(Y_test[2])))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 3)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
3/3 - 0s - loss: 0.1040 - categorical_accuracy: 1.0000
pred: 2, actual: 2
pred: 2, actual: 2
pred: 2, actual: 2
